In [21]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
import os

def function1(filename):

    loader = TextLoader(filename)
    docs = loader.load()

    # Define prompt
    prompt_template = """Write a detailed and consise summary which captures every key point of the following parts of the Planning and Building Act:
    "{text}"
    DETAILED SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Define LLM chain
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Define StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(
        llm_chain=llm_chain, document_variable_name="text"
    )

    # Create a new file with the _summary.txt naming convention
    #summary_filename = os.path.join(os.path.dirname(filename), os.path.basename(os.path.dirname(filename)) + "_summary.txt")
    summary_filename = os.path.join(filename[:-4] + "_summary.txt")
    print(summary_filename)
    with open(summary_filename, 'w') as f:
        # Your logic to write to the file
        f.write(stuff_chain.run(docs))  # Example text

def function2(chapter_dir, filenames):

    loader = DirectoryLoader(chapter_dir)
    docs = loader.load()

    prompt_template = """Write a long detailed summary which captures every key point of the following parts of the Planning and Building Act:
    {text}
    DETAILED SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    refine_template = (
        "Your job is to produce a final long detailed summary which captures every key point\n"
        "We have provided an existing long and detailed summary up to a certain point: {existing_answer}\n"
        "We have the opportunity to refine the existing long and detailed summary"
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new context, refine the original long and detailed summary."
        "If the context isn't useful, return the original long and detailed summary."
    )
    refine_prompt = PromptTemplate.from_template(refine_template)
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
    chain = load_summarize_chain(
        llm=llm,
        chain_type="refine",
        question_prompt=prompt,
        refine_prompt=refine_prompt,
        return_intermediate_steps=True,
        input_key="input_documents",
        output_key="output_text",
    )
    result = chain({"input_documents": docs}, return_only_outputs=True)



    # Create a new file with the _summary.txt naming convention
    summary_filename = os.path.join(chapter_dir + "_summary.txt")
    with open(summary_filename, 'w') as f:
        # Your logic to write to the file
        f.write(result['output_text'])  # Example text

def process_parts():
    base_dir = r"C:\Users\fredr\AI\PlanAndBuild\parts"
    # Loop through all the files in the parts directory
    parts = os.listdir(base_dir)
    for part in parts:
        part_dir = os.path.join(base_dir, part)
        function1(part_dir)

In [18]:
import os
base_dir = r"C:\Users\fredr\AI\PlanAndBuild\parts"
# Loop through all the files in the parts directory
parts = os.listdir(base_dir)
filename = os.path.join(base_dir, parts[0])
print(filename)
# remove the .txt from the filename
os.path.join(filename[:-4] + "_summary.txt")

C:\Users\fredr\AI\PlanAndBuild\parts\III_Regional_planning.txt


'C:\\Users\\fredr\\AI\\PlanAndBuild\\parts\\III_Regional_planning_summary.txt'

In [22]:
# Call the main function to process chapters
process_parts()

C:\Users\fredr\AI\PlanAndBuild\parts\III_Regional_planning_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\III_Regional_planning_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\II_National_planning_functions_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\II_National_planning_functions_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\IV_Municipal_planning_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\IV_Municipal_planning_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\I _Introductory_provisions_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\I _Introductory_provisions_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\parts_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\parts_summary_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\Part_II__The_planning_part_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\Part_II__The_planning_part_summary_summary.txt
C:\Users\fredr\AI\PlanAndBuild\parts\Part_I__General_provisions_summary.txt
C:

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 18554 tokens. Please reduce the length of the messages.

In [ ]:
# For every .txt files ending with _summary.txt in "C:\Users\fredr\AI\phd\chapters", print the contents of each file.

dir = r'C:\Users\fredr\AI\phd\chapters'
for filename in os.listdir(dir):
    if filename.endswith("_summary.txt"):
        with open(os.path.join(dir, filename), 'r') as f:
            print("\n Chapter summary: " + filename + "\n")
            print(f.read())
